# CAPSTONE PROJECT: BATTLE OF THE NEIGHBORHOODS

## Analyzing Food Place in Bali

### I. Purpose

This document provides the details of my final peer reviewed assignment for the IBM Data Science Professional Certificate program – Coursera Capstone.

### II. Introduction

Mostly for those who first time visiting Bali, it will be hard to find the best location to stay where you may find good foods within a walking distance from where you stay. Plenty of dining options ranging from a casual cafe brunch, quality local coffee, enticing patisseries and fresh seafood. Regardless of your budget, it’s possible to have something different and exciting every meal but how can we decide the best location especially for the foodies.  

Most people plan their first trip to Bali for the sun and beaches but I would definitely go back for the amazing food. Known for its surf worthy beaches, spiritual retreats and endless outdoor activities, Bali is also a foodie heaven if you know where to look.

My data visualisation project hopes to provide planners with a cockpit view of the local food landscape, by mapping the distribution of the different cuisines in Bali, to help identify ethnic food enclaves, trace the popularity trends of specific local foods in Bali, as well as to understand how price elasticity of local food impacts it's economic accessibility and review ratings.

### III. Data

In order to accomplish this goal, latitudes and longitudes of Bali is required. Wikipedia offers a simple, accurate and up-to-date database of the world's cities and their locations. From this data, I need to select the city and using their latitude and longitude values, I need to explore the venues with "restaurant" section around these city by utilizing Foursquare API. The following information are retrieved on the first query:

-Venue ID 

-Venue Name

-Coordinates : Latitude and Longitude

-Category Name

Another venue query will be performed to retrieve venue ratings for each location. Note that rating information is a paid service from FourSquare and we are limited to only 50 queries per day. With this constraint, we limit the category analysis with only one type for this demo. I will try to retrieve as many ratings as possible for each retrieved venue ID.

### Mapping Data - Bali Map

In [1]:
import os,sys
import urllib
import requests 
import json
from urllib.request import urlopen
import pandas as pd
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
import folium
import math

In [2]:
CLIENT_ID = '500AOC1YJOHU2PUF2ACO00CWUDPG5CZJT33IAEUXNNBLRB5K' # your Foursquare ID
CLIENT_SECRET = 'CQ3DBPGGUW22MD5KTVYU5N4I5SDXNZV0PVSLH0QXREPSI0JH' # your Foursquare Secret
VERSION = '20180605'
QUERY = 'food'
LIMIT = 10000
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 500AOC1YJOHU2PUF2ACO00CWUDPG5CZJT33IAEUXNNBLRB5K
CLIENT_SECRET:CQ3DBPGGUW22MD5KTVYU5N4I5SDXNZV0PVSLH0QXREPSI0JH


In [3]:
# get the location of the city
def getlocation(address):
    geolocator = Nominatim()
    location = geolocator.geocode(address)
    latitude = 0.0
    latitude = location.latitude
    longitude = 0.0
    longitude = location.longitude
    print('The geograpical coordinate of ',address,'are {}, {}.'.format(latitude, longitude))
    return latitude,longitude

In [4]:
# get the dataframe of the place using foursqare api
def getNearbyVenues_new(name, latitudes, longitudes, radius=100000):
    
    venues_list=[]
#    for name, lat, lng in zip(names, latitudes, longitudes):
#        print(name)
            
        # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        latitudes, 
        longitudes, 
        radius, 
        100000)
            
        # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
       
        # return only relevant information for each nearby venue
    venues_list.append([(
        name, 
        v['venue']['name'], 
        v['venue']['location']['lat'], 
        v['venue']['location']['lng'],  
        v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Location','Venue','Latitude','Longitude','Category']    
    return(nearby_venues)

In [5]:
# color the place in Map
def mapMarkers(map_name,dfs_data):  # add markers to map
    for lat, lng, label in zip(dfs_data['Latitude'], dfs_data['Longitude'], dfs_data['Venue']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7).add_to(map_name)

In [6]:
address_bali = 'Bali,ID'
#location_sh = getlocation
latitude_bali = -8.719266
longitude_bali = 115.168640
print('The geograpical coordinate of ',address_bali,'are {} {} .'.format(latitude_bali,longitude_bali))

The geograpical coordinate of  Bali,ID are -8.719266 115.16864 .


In [7]:
map = folium.Map(location= [latitude_bali,longitude_bali], tiles='Stamen Terrain',zoom_start=8)

folium.Marker(location=[latitude_bali,longitude_bali], popup='BALI').add_to(map)
folium.CircleMarker(location=[latitude_bali,longitude_bali], radius=10,
popup='BALI', color='#3186cc',fill_color='#3186cc').add_to(map)

map

In [8]:
map.save('map.html')

### Getting Restaurant Venues in Bali

In [9]:
dfs = getNearbyVenues_new('Bali', latitude_bali,longitude_bali)
dfs.head()

,Location,Venue,Latitude,Longitude,Category
0,Bali,Odysseys Surf School,-8.720849,115.169901,Surf Spot
1,Bali,Sheraton Bali Kuta Resort,-8.717966,115.169126,Hotel
2,Bali,The Anvaya Beach Resort,-8.732262,115.165786,Resort
3,Bali,Hotel Indigo Bali Seminyak Beach,-8.695403,115.162985,Resort
4,Bali,Beachwalk,-8.716909,115.169234,Shopping Mall


In [10]:
map = folium.Map(location=[latitude_bali,longitude_bali], zoom_start=13)
mapMarkers(map,dfs)
map

In [11]:
map.save('map.html')

In [12]:
dfs.shape

(100, 5)

In [13]:
df1 = dfs.set_index('Category').filter(like='Indonesian', axis=0)

In [14]:
df1.head(100)

,Location,Venue,Latitude,Longitude
Category,,,,
Indonesian Restaurant,Bali,Bambu,-8.680180,115.153213
Indonesian Restaurant,Bali,Merah Putih Restaurant,-8.676683,115.154592
Indonesian Restaurant,Bali,Warung Pojok,-8.640958,115.221910
Indonesian Restaurant,Bali,Nook Restaurant,-8.669573,115.161263
Indonesian Restaurant,Bali,Warung Sate Plecing Mak Lukluk,-8.673840,115.243818
Indonesian Restaurant,Bali,Warung Bu Kris,-8.640976,115.208750


In [15]:
map = folium.Map(location=[latitude_bali,longitude_bali], zoom_start=13)
mapMarkers(map,df1)
map

### Result Discussion

Popular Indonesian restaurants are limited and scattered around the Bali island. There are 6 Indonesian restaurants identified. We can conclude that similar restaurants (i.e Indonesian restaurants) are not concentrated in one area. Visitors might experience a hard time to travel around. 

### Conclusion

From the table extracted earlier, we can know that Foursquare is not a very popular application to use in Bali. 
Given that we can only rely on Foursquare API data, we can conclude that around Bali city area there are only 5 Indonesian
restaurant, which logically it can not be true. From this exercise I can say that Foursquare is not a good API provider to use to analyse locations in Bali effectively. 